<a href="https://colab.research.google.com/github/brandontan99/Self_Love_App/blob/master/OneVsRest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/brandontan99/Self_Love_App.git
%cd Self_Love_App

Cloning into 'Self_Love_App'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 196 (delta 109), reused 50 (delta 22), pack-reused 0
Receiving objects: 100% (196/196), 1.88 MiB | 9.26 MiB/s, done.
Resolving deltas: 100% (109/109), done.
/content/Self_Love_App


In [ ]:
import sys 
sys.path.append("./preprocessing")
sys.path.append("./utils")
from data_cleaning import *
from Data_Normalization import *
from feature_selection import *

import pandas as pd

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
import statistics
import joblib
from copy import deepcopy
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.model_selection import GridSearchCV

In [ ]:
    #Feature selection
    from sklearn.preprocessing import LabelEncoder
    from sklearn.feature_selection import chi2, SelectKBest
    # to do the chi2 analysis
    df = pd.read_csv("WID3006 ML Questionnaire.csv")
    df = data_cleaning(df)
    # label encoding is used instead of one-hot encoding because one-hot encoding will create many features for one question
    # but label encoding is able to perform chi-square test between a single question as a whole and the hobby  
    x_df, y_df = label_encoding(df)
    # only evaluate on train data to prevent data leakage
    x_train_label_encoded, _, y_train, _ = train_test_split(x_df.to_numpy(), y_df.to_numpy(), test_size=0.2, random_state=1)
    chi2_result = chi2_analysis(x_train_label_encoded, x_df, y_train)

    # to do Recursive Feature Elimination(rfe) feature selection
    df = pd.read_csv("WID3006 ML Questionnaire.csv")
    df = data_cleaning(df)
    df = data_encoding(df)
    df_norm = data_normalization(df)
    x = df_norm.iloc[:, :64]
    y = df_norm.iloc[:, 64:]
    x_numpy, y_numpy = x.to_numpy(), y.to_numpy()
    x_train, x_test, y_train, y_test = train_test_split(x_numpy, y_numpy, test_size=0.2, random_state=1)
    rfe_result = rfe_cv(x_train, y_train, x.columns, y.columns, LogisticRegression())

    # to pick the selected best questions as features
    # Eg: choose the best 13 features as below
    df = pd.read_csv("WID3006 ML Questionnaire.csv")
    df = data_cleaning(df)
    df = data_encoding(df)
    df_norm = data_normalization(df)
    # choose either chi2 or rfe 
    best_k_features = select_best_k_features(chi2_result, k=13) # choose chi2_result 
    best_k_features = select_best_k_features(rfe_result, k=13) # choose rfe_analysis
    x = filter_features(best_k_features, df_norm)
    x_train, x_test,y_train,y_test = train_test_split(x,y, random_state=1, test_size=0.25, shuffle=True)


Hobby: Sports and Outdoors
Best number of features: 11
Score: 0.6797849462365592
1 Gender: _Male
21 Choose a pet which you prefer to keep._Horse
25 Choose a pet which you prefer to keep._Tortoise
40 What is your favorite color?_Blue
50 What is your learning style? (Pick one that benefit you the most)_By reading an e-Book
52 Do you enjoy challenges?
55 Would you prefer to engage your brain more than your body?
57 Are you a perfectionist?
58 Are you a trusting person?
59 Do you have lot of patience?
64 How confident are you in your own abilities?

Hobby: Games
Best number of features: 10
Score: 0.5944086021505377
1 Gender: _Male
4 What is your current occupation?_University student
11 I prefer to spend my money on...._The latest fashion
22 Choose a pet which you prefer to keep._I'm not a pet person
31 What do you worry more about the most?_Money
33 What do you worry more about the most?_Your family and friends
35 When you retire, you'd like to live..._Exactly where I live now
42 What is

In [ ]:
#Define all the possible classifiers.
parameters = [
    {
        'classifier': [LogisticRegression(solver='sag')]
    },
    {
        'classifier': [MultinomialNB(fit_prior=True, class_prior=None)]
    },
    {
        'classifier': [LinearSVC()]
    }
]

#Choose the best parameter for the model.
clf = GridSearchCV(BinaryRelevance(), parameters, scoring='f1_micro')
clf.fit(x_train, y_train)

print (clf.best_params_, clf.best_score_)


{'classifier': MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)} 0.5475998015895553


In [ ]:
#The best model with some tuning.
classifier = BinaryRelevance(
    classifier = LogisticRegression(solver='sag'),
    require_dense = [False, True]
)


classifier.fit(x_train, y_train)
print(classifier.model_count_)
prediction=classifier.predict(x_test)
f1=sklearn.metrics.f1_score(y_test, prediction,average="micro")
f1

8


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

0.5849802371541503